Functionalities relating to applyCl. 

# balanceFolder

## Log scale file sizes

In [2]:
base = "~/repos/labs/k1lib/k1lib/cli/test/applyCl.balance"; #base = "~/ssd2/test"
applyCl.cmd(f"rm -r {base}")
applyCl.cmd(f"mkdir -p {base}")
torch.loglinspace(10, 1e5, 20).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx}.txt")) | deref();

In [3]:
#export
def getIr(base): return None | applyCl.aS(lambda: ls(base) | iden() & apply(os.path.getsize) | transpose() | deref()) | ungroup(begin=True) | insertIdColumn(True) | deref()

In [4]:
ir = getIr(base); ir | head(2)

[[0,
  '244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732',
  '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/15.txt',
  14385],
 [1,
  '244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732',
  '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/0.txt',
  11]]

ir = intermediate representation: `[id, node id, url, size]`

In [5]:
#export
def normalize(d):
    d = d | deref(); s = d | cut(1) | toSum()
    return d | apply(op()/s, 1) | sort(0, False) | deref()
@lru_cache
def statsCpu():
    cpu = None | applyCl.aS(applyCl.cpu) | sort(0, False) | deref()
    cpuF = None | applyCl.aS(applyCl.cpu) | aS(normalize); cpuF # "cpuF" = cpu fraction. List[nodeId, cpu fraction]
    return [cpu, cpuF]
@lru_cache
def statsNodeId(): return applyCl.nodeIds()
_statsS1 = statsNodeId() | apply(wrapList() | insert(0, False)) | toDict()
def stats(ir):
    cpu, cpuF = statsCpu()
    # size fraction. List[nodeId, size fraction]
    sizeF = normalize({**_statsS1, **ir | groupBy(1, True) | apply(cut(2) | toSum(), 1) | toDict()}.items()); sizeF
    return *statsCpu(), sizeF
def scores(ir): cpu, cpuF, sizeF = stats(ir); return [cpuF, sizeF] | cut(1).all() | transpose() | ~apply(lambda c,s: s-c) | deref() # negpos
def score(ir): return scores(ir) | apply(lambda x: abs(x)**2) | toSum() # pos

In [6]:
#export
def move(ir, nA:str, nB:str, idx:int):
    ir1 = ir | deref(); ir1[idx][1] = nB; a = ir1 | pretty() | join('\n'); return ir1
def optimize(ir):
    cpu, cpuF, sizeF = stats(ir); scs = scores(ir); a = np.argmax(scs); b = np.argmin(scs)
    # print(scs, a, b)
    nodeIds = applyCl.nodeIds() | sort(None, False) | deref()
    files = {**nodeIds | apply(wrapList() | insert([], False)) | toDict(), **ir | groupBy(1, True) | toDict()}.items() | sort(0, False) | deref()
    fA = files[a]; fB = files[b] # files A. [nodeId, List[idx, url, size]]
    # print(fA); print(fB)
    nA = nodeIds[a]; nB = nodeIds[b] # A node id
    sA, sB = [fA, fB] | apply(op()[1] | cut(2) | deref() | aS(np.array, dtype=int)) # file sizes A
    # print(f"sA: {sA} {sB}")
    spA = sA.sum() - sA; spB = sB.sum() + sA # sum prime A, array[files]
    sp = spA + spB; sfA = spA/sp; sfB = spB/sp
    cA = cpu[a][1]; cB = cpu[b][1] # cpu A
    c = cA + cB; cfA = cA/c; cfB = cB/c # cpu fraction A
    # print(f"sfA: {sfA}, cfA: {cfA}")
    exp = 5 # intuition says that exp should be even. But that doesn't work. Odd values work tho, but I have no idea why
    idx = fA[1][((sfA-cfA)**exp + (sfB-cfB)**exp).argmin()][0]
    ir2 = move(ir, nA, nB, idx)
    return ir2, [nA, nB, idx, score(ir2)]

In [7]:
#export
def traj(ir, maxSteps=20):
    sc = score(ir); aux = None; auxs = []; irs = []
    for i in range(maxSteps):
        ir, aux = optimize(ir)
        if aux[3] > sc:
            #irs.append(ir); auxs.append(aux)
            break
        irs.append(ir); auxs.append(aux); sc = aux[3]
    return irs, auxs

In [8]:
traj(ir)[1] | display(None)

244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d   0    0.298215197046367      
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0   3    0.2914574353376609     
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0   14   0.27410841688253984    
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0   1    0.2740527139855046     
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0   4    0.07415834452522455    
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d   2    0.07185421104196338    
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d   

In [9]:
#export
def collapse(it):
    a, b = it | rows(0, -1)
    c = [a[0], b[1], a[2], b[3]]
    if c[0] == c[1]: return []
    return [c]
def traj2(ir):
    idx2FileName = ir | apply(lambda arr: [arr[0], arr[2]]) | toDict()
    a = traj(ir)[1] | groupBy(2) | filt(lambda x: len(x) > 1).split() | (apply(collapse)) + iden() | joinStreams(2) | deref()
    return a | lookup(idx2FileName, 2) | deref()
def moveFile(fileName:str, destNodeId:str, timeout=60): # executed on remotes
    fn = os.path.expanduser(fileName); dirname = os.path.dirname(fn)
    [destNodeId] | applyCl.aS(lambda: None | cmd(f"mkdir -p {dirname}; rm -f {fn}") | deref(), timeout=timeout) | deref()
    for chunk in cat(fn, False, True): [destNodeId] | applyCl.aS(lambda: chunk >> file(fn), timeout=timeout) | deref()
    None | cmd(f"rm {fn}") | deref()
def balance(base): traj2(getIr(base)) | apply(lambda arr: [arr[0], arr]) | ~applyCl(lambda a,b,fn,sc: moveFile(fn, b), pre=True, timeout=60) | deref()

In [10]:
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 0],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 260307],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 0]]

In [11]:
balance(base)

In [12]:
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 66231],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 117508],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 76568]]

In [13]:
assert None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | cut(1) | toProd() > 0

In [14]:
torch.loglinspace(10, 1e5, 20).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx+21}.txt")) | deref();

In [15]:
balance(base)

In [16]:
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 168485],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 275561],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 76568]]

## Linear scale file sizes

In [17]:
base = "~/repos/labs/k1lib/k1lib/cli/test/applyCl.balance"; #base = "~/ssd2/test"
applyCl.cmd(f"rm -r {base}"); applyCl.cmd(f"mkdir -p {base}")
torch.linspace(1e4, 1e5, 20).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx}.txt")) | deref();
balance(base); None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 244740],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 563690],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 291581]]

In [18]:
torch.linspace(1e4, 1e5, 20).int() | apply(lambda x: "x"*x) | insertIdColumn() | ~apply(lambda idx, contents: contents | file(f"{base}/{idx+20}.txt")) | deref();
balance(base); None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | deref()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 597901],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 1091065],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 511056]]

In [19]:
None | applyCl.aS(lambda: ls(base) | apply(os.path.getsize) | toSum()) | cut(1) | toSum() | deref()

2200022

In [20]:
torch.linspace(1e4, 1e5, 20).int() | toSum() | op()*2+40

tensor(2200022)

In [21]:
None | applyCl.aS(lambda: ls(base)) | deref()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0',
  ['/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/13.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/39.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/31.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/17.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/27.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/0.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/18.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/3.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/4.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/2.txt',
   '/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance/29.txt']],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732',
  ['/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.balance

Looks about right.

# Distributed download

In [22]:
from k1lib.imports import *

In [23]:
#export
def getSize(url):
    for i in range(10):
        try: return requests.head(url, timeout=3).headers.items() | apply(op().lower(), 0) | toDict() | op()["content-length"].ab_int()
        except Exception as e:
            if i == 9: raise Exception(f"Can't get size of file")
class NoPartialContent(Exception): pass
def getChunk(url:str, sB:int, eB:int, timeout:float) -> bytes:
    for i in range(10):
        try: res = requests.get(url, headers={"Range": f"bytes={sB}-{eB}"}, timeout=timeout)
        except Exception as e:
            if i == 9: raise Exception(f"Can't get file chunk")
            continue
        if res.status_code != 206: raise NoPartialContent("Server doesn't allow partial downloads at this particular url")
        return res.content
def getChunks(url:str, sB:int, eB:int, chunkSize=None, chunkTimeout:float=10) -> List[bytes]:
    """Grabs bytes from sB to eB in chunks"""
    chunkSize = chunkSize or settings.cli.cat.chunkSize
    return range(sB, eB+1) | batched(chunkSize, True) | apply(lambda r: getChunk(url, r.start, r.stop-1, chunkTimeout))
def download(url:str, folder:str, merge:bool=False, timeout=120, chunkTimeout=5):
    getChunk(url, 0, 1, 10) # try to see if server accepts partial downloads first
    folder = os.path.expanduser(folder); dirname = os.path.dirname(folder)
    if merge: destFile = folder; folder = b"" | file(); None | cmd(f"rm -rf {folder}") | ignore(); None | cmd(f"mkdir -p {folder}") | ignore()
    applyCl.cmd(f"rm -rf {folder}"); applyCl.cmd(f"mkdir -p {folder}"); size = getSize(url)
    cpus = None | applyCl.aS(lambda: applyCl.cpu()) | deref(); n = cpus | cut(1) | toSum()
    tasks = [cpus | ~apply(lambda x,y: [x]*y) | joinStreams(), range(size) | splitW(*[1]*n)] | transpose() | insertIdColumn(True, False) | ~apply(lambda x,y,z: [x,[y,f"{folder}/{z}.bin"]]) | deref(1)
    tasks | ~applyCl(lambda r, fn: getChunks(url, r.start, r.stop-1, None, chunkTimeout) | file(fn), pre=True, timeout=timeout) | deref()
    if merge:
        None | cmd(f"rm -rf {destFile}") | deref()
        None | cmd(f"mkdir -p {dirname}") | deref()
        None | applyCl.aS(lambda: ls(folder)) | ungroup(True, True) | deref() | sortF(op().split(".bin")[0].split("/")[-1].ab_int(), 1) | applyCl(cat(text=False), pre=True) | cut(1) | file(destFile)
        None | cmd(f"rm -rf {folder}") | deref()

In [24]:
url = "https://vim.kelvinho.org"
fn = "~/repos/labs/k1lib/k1lib/cli/test/applyCl.download" # file/folder name
download(url, fn, True)       # will download distributedly and dump file fragments into the folder fn

# Advanced split file

In [1]:
from k1lib.imports import *
base = "/home/kelvin/repos/labs/k1lib/k1lib/cli/test/applyCl.advSplit"
fn = f"{base}/a.txt"
def reset():
    applyCl.cmd(f"rm -r {base}"); applyCl.cmd(f"mkdir -p {base}")
    applyCl.nodeIds() | head(2) | applyCl.aS(lambda: "0123456789\n"*10000 | file(fn)) | deref()
def status(): return None | applyCl.aS(lambda: fn | (tryout(0) | cat(text=False) | shape(0))) | deref()
nodeIds = applyCl.nodeIds(); reset(); status()

2023-05-15 11:03:04,854	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 192.168.1.35:6379...
2023-05-15 11:03:04,858	INFO worker.py:1544 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 110001],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 110001],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 0]]

In [2]:
nse = [applyCl.nodeIds()[:2], [0, 1234], [4567, 12345]] | transpose() | apply(lambda arr: [arr[0], arr[1:]]) | deref(); nse

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', [0, 4567]],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', [1234, 12345]]]

In [3]:
#export
def a_transfer(fn, nse, nodeB, rpF:callable=iden()):
    """Transfers a lot of blocks from a bunch of nodes to nodeB. Does not delete from those node though

nse = List[nodeAId, [sB, eB]]

Runs on driver process, blocks, so better use applyTh outside of this

:param rpF: ray progress function"""
    blockSize = settings.cli.cat.chunkSize
    def inner():
        totalBytes = nse | cut(1) | ~apply(lambda x,y:y-x) | toSum(); currentByte = 0
        for chunk in nse | ~apply(lambda x, y: range(x, y) | batched(blockSize, True) | apply("[x.start, x.stop]"), 1) | ungroup(True, True) | deref()\
            | ~applyCl(lambda sB, eB: cat(fn, False, sB=sB, eB=eB), pre=True) | cut(1):
            chunk >> file(fn); currentByte += len(chunk); rpF(currentByte/totalBytes)
    [nodeB] | applyCl.aS(inner) | item()

In [4]:
%%time
reset()
a_transfer(fn, nse, applyCl.nodeIds()[2])
assert None | applyCl.aS(lambda: cat(fn, False) | shape(0)) | deref() | op()[2][1] == 4567+11111

CPU times: user 15.9 ms, sys: 3.1 ms, total: 19 ms
Wall time: 148 ms


In [5]:
#export
def decommission(fn:str, nodeAs:List[str], nodeBs:List[str], rS=iden()):
    """Spreads out a particular file in nodeAs to all nodeBs, to prepare
to decomission nodeAs. The 2 sets should be mutually exclusive

:param rS: instance of refineSeek"""
    nodeAs, nodeBs = [nodeAs, nodeBs] | deref()
    if len(nodeAs) == 0: return
    if len(nodeBs) == 0: raise Exception("Unsupported configuration! Trying to move data from A+B to C+D. Has to have some shared nodes, like moving data from A+B+C to B+C+D. This is not a fundamental limitation, but just can't be done with the current architecture. Might be fixed in the future.")
    # some initial metadata
    nodeIds = applyCl.nodeIds(); nodeId_cpu = nodeIds | applyCl.aS(lambda: applyCl.cpu()) | deref(); nodeId2Cpu = nodeId_cpu | toDict()
    ws = nodeId_cpu | inSet(nodeBs, 0) | cut(1) | deref() # weights to split files on nodeAs into
    # splitting file on nodeAs into chunks first, to plan things out
    a = nodeAs | applyCl.aS(lambda: fn | splitSeek(ws=ws) | rS | window(2) | deref() | insertColumn(nodeBs) | insert(applyCl.nodeId()).all() | deref()) | cut(1) | joinStreams() | deref()
    # actually transferring chunks
    with ray.progress(a | groupBy(1) | shape(0), "Decommissioning") as rp:
        c = b = a | groupBy(1, True) | apply(iden() + apply(lambda arr: [arr[0], arr[1:]]) | reverse() | insert(fn)) | deref()
        enumerate(c) | applyTh(~aS(lambda idx, e: a_transfer(*e, rpF=aS(lambda p: ray.get(rp.update.remote(idx, p)))))) | deref()
    # deleting files from nodeAs
    nodeAs | applyCl.aS(lambda: None | cmd(f"rm -rf {fn}") | ignore()) | deref()

In [6]:
reset(); status() | display()
decommission(fn, *applyCl.nodeIds() | splitW(1, 2))
assert status() | ~head(1) | cut(1) | toProd() == 6723517725

867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0   110001   
244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732   110001   
4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d   0        


In [7]:
status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 0],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 183327],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 36675]]

In [8]:
nAs = ["867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0", "244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732"]
nBs = ["4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d"]
ns = [*nAs, *nBs]

In [36]:
#export
def spreadOut(fn:str, nAs:List[str], nBs:List[str]):
    """Spreads out a file from nodes A to B, where B fully contains A (no decomissioning).
A and B should be mutually exclusive. Initial nodes are A, final nodes are A + B"""
    nAs, nBs = [nAs, nBs] | deref()
    if len(nBs) == 0: return # no need to spread out
    nBs | applyCl.aS(lambda: None | cmd(f"mkdir -p {os.path.dirname(fn)}") | deref()) | deref()
    nBs | applyCl.aS(lambda: None | cmd(f"rm -rf {fn}") | deref()) | deref()
    # some initial metadata
    nodeIds = applyCl.nodeIds(); nodeId_cpu = nodeIds | applyCl.aS(lambda: applyCl.cpu()) | deref(); nodeId2Cpu = nodeId_cpu | toDict()
    sizes = nAs | applyCl.aS(lambda: os.path.getsize(fn) if os.path.exists(fn) else 0) | deref(); totalSize = sizes | cut(1) | toSum()
    ns = [*nAs, *nBs]; totalCpu = ns | lookup(nodeId2Cpu) | toSum(); bytePerCpu = totalSize/totalCpu; wsB = nBs | lookup(nodeId2Cpu) | deref()
    # prepares segments and metadata, List[nodeId, [sB, eB]], where sB and eB are the ranges of nAs that they're willing to share
    sizePost = sizes | ~apply(lambda idx, size: [idx, nodeId2Cpu[idx]/totalCpu*totalSize/size]) | deref()
    invalidNodes = sizePost | ~filt(lambda x: 0 <= x <= 1, 1) | cut(0) | deref()
    if len(invalidNodes) > 0: raise Exception(f"Unsupported configuration! These nodes have too little data to share: {invalidNodes}. This couldn't have happen using applyCl alone. Data is not corrupted, but you'll have to combine data from all files into 1 and spread them back out again.")
    inter = sizePost | ~apply(lambda idx, x: [idx, [x, 1-x]]) | applyCl(lambda ws: fn | splitSeek(ws=ws) | ~head(1), pre=True) | deref() | filt(~aS(lambda x,y: y-x>0), 1) | deref()
    # actually transferring data to new nodes
    meta = inter | apply(~aS(range) | splitW(*wsB) | apply(wrapList()) | insertColumn(nBs) | deref(1), 1) | ungroup(begin=True) | apply("[x.start, x.stop]", 2) | groupBy(1, True) | deref()
    with ray.progress(len(meta), "Transferring data to new nodes") as rp:
        meta | insertIdColumn(True) | applyTh(~aS(lambda idx, nB, nse: a_transfer(fn, nse, nB, rpF=aS(lambda p: ray.get(rp.update.remote(idx, p)))))) | deref()
    # truncates the files in nAs nodes
    inter | ~apply(lambda idx,se: [idx,se[0]]) | applyCl(lambda sB: open(fn, 'a').truncate(sB), pre=True) | deref()

In [37]:
#export
def balanceFile(fn:str, nAs:List[str]=None, nBs:List[str]=None):
    fn = os.path.expanduser(fn)
    if nAs is None: nAs = None | applyCl.aS(lambda: os.path.exists(fn)) | filt(op(), 1) | cut(0) | deref()
    if nBs is None: nBs = applyCl.nodeIds()
    decommission(fn, *nAs | inSet(nBs).split() | reverse())
    spreadOut(fn, *nBs | inSet(nAs).split())

In [35]:
None | applyCl.aS(lambda: applyCl.cpu()) | deref()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 8],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 16],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 8]]

In [30]:
reset(); "0123456789\n"*5000 >> file(fn); status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 110001],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 165002],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 0]]

In [31]:
balanceFile(fn)

In [32]:
status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 68750],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 137501],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 68752]]

In [33]:
balanceFile(fn, nBs=["867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0", "4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d"])

In [34]:
status()

[['867dc6c8f88045aa165af30a64125aee311f1f64a32f1c43c6ed4dc0', 137500],
 ['244f4e340443f74ab302822376195329f45858b65998b4a77f5f4732', 0],
 ['4ffae9ae502fde1a76a7b54a0146c943be6d0ac9bba8f7c3e07fdb4d', 137503]]

In [39]:
!../../export.py cli/_applyCl

Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/cli/../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.3.7.10
Uninstalling k1lib-1.3.7.10:
  Successfully uninstalled k1lib-1.3.7.10
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/